<a href="https://colab.research.google.com/github/perlatomdpi/NLP/blob/main/BERT_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BERT Tokenizer**
Typical elasticSearch, solR use TF-IDF. <br>
The drawback is that it doesn't a semantic match.

